# OpenURL Resolver Testing

This notebook demonstrates how to use the OpenURL resolvers in SciTeX Scholar.

## Setup

In [ ]:
from scitex.scholar.open_url import OpenURLResolver, OpenURLResolverWithZenRows
from scitex.scholar.auth import AuthenticationManager
import os
from scitex import logging

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Initialize Authentication

In [ ]:
# Initialize authentication manager
auth_manager = AuthenticationManager(
    email_openathens=os.getenv("SCITEX_SCHOLAR_OPENATHENS_EMAIL")
)

## Test DOIs

In [ ]:
# DOIs to test
dois = [
    "10.1002/hipo.22488",           # Wiley
    "10.1038/nature12373",          # Nature
    "10.1016/j.neuron.2018.01.048", # Elsevier
    "10.1126/science.1172133",      # Science
    "10.1073/pnas.0608765104",      # PNAS
]

## Method 1: ZenRows Resolver (Fast, Anti-bot)

In [ ]:
# Initialize ZenRows resolver
zenrows_resolver = OpenURLResolverWithZenRows(
    auth_manager,
    os.getenv("SCITEX_SCHOLAR_OPENURL_RESOLVER_URL"),
    os.getenv("SCITEX_SCHOLAR_ZENROWS_API_KEY")
)

In [ ]:
# Test single DOI first
result_0 = zenrows_resolver._resolve_single(doi=dois[0])
print(f"Single DOI test: {dois[0]}")
print(f"Success: {result_0.get('success')}")
print(f"URL: {result_0.get('final_url')}")

In [ ]:
# Resolve all DOIs
print("Resolving all DOIs with ZenRows...")
results = zenrows_resolver.resolve(dois)

In [ ]:
# Analyze results
for doi, result in zip(dois, results):
    if result:
        print(f"\n{'✓' if result.get('success') else '✗'} {doi}")
        print(f"  URL: {result.get('final_url', 'N/A')}")
        
        # Get detailed access info
        if hasattr(zenrows_resolver, '_last_page_content') and zenrows_resolver._last_page_content:
            details = zenrows_resolver.get_access_details(
                zenrows_resolver._last_page_content,
                result.get('final_url', ''),
                doi
            )
            if not details['has_access']:
                print(f"  No access: {details['likely_reason']}")

## Method 2: Browser Resolver (For Authenticated Access)

Use this method when you need authenticated access to paywalled content.

In [ ]:
# Uncomment to use browser-based resolver
# browser_resolver = OpenURLResolver(
#     auth_manager,
#     os.getenv("SCITEX_SCHOLAR_OPENURL_RESOLVER_URL")
# )

# # Make browser visible to see what's happening
# browser_resolver.browser.set_visible(True)

# # Resolve DOIs
# browser_results = browser_resolver.resolve(dois)

## Summary and Recommendations

### When to use ZenRows Resolver:
- Checking if content is open access
- High-volume processing
- Bypassing anti-bot measures
- Speed is important

### When to use Browser Resolver:
- Need authenticated access to paywalled content
- Have institutional credentials
- Quality over speed
- Complex authentication flows